In [1]:
!pip -q install transformers  torch
!pip -q install peft
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
from peft import PeftModel
import os
import time
import logging
from pathlib import Path

In [3]:
LORA_ADAPTER_PATH = "mahmuuud/QA-EnergyAI-2025-07-29_09.16.39"

# Base model name
BASE_MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

# Output paths
OUTPUT_PATH = "./optimized_model"

In [ ]:
from huggingface_hub import login
login(token="")

In [5]:
# 1) load base in 4bit + your adapter
base_model  = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16
)
model = PeftModel.from_pretrained(base_model, LORA_ADAPTER_PATH)

merged_model = model.merge_and_unload()

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

In [6]:
!mkdir ./merged_model_bf16

In [7]:
merged_model.save_pretrained("./merged_model_bf16")
tokenizer.save_pretrained("./merged_model_bf16")

NameError: name 'tokenizer' is not defined

In [8]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME , trust_remote_code=True)
tokenizer.save_pretrained("./merged_model_bf16")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

('./merged_model_bf16/tokenizer_config.json',
 './merged_model_bf16/special_tokens_map.json',
 './merged_model_bf16/chat_template.jinja',
 './merged_model_bf16/tokenizer.json')

In [9]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 57717, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 57717 (delta 145), reused 52 (delta 52), pack-reused 57504 (from 5)
Receiving objects: 100% (57717/57717), 137.00 MiB | 29.72 MiB/s, done.
Resolving deltas: 100% (41798/41798), done.


In [10]:
!mkdir models

In [11]:
!python llama.cpp/convert_hf_to_gguf.py ./merged_model_bf16 --outfile models/llama_3.2_FP16.gguf

INFO:hf-to-gguf:Loading model: merged_model_bf16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {3072, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {8192, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {3072}
INFO:hf-to-gguf:blk.0.at

In [12]:
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP:

In [13]:
!cd llama.cpp/build/bin && ./llama-quantize /content/models/llama_3.2_FP16.gguf /content/models/llama_3.2-Q4_K_M.gguf q4_K_M

main: build = 6029 (a118d802)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/models/llama_3.2_FP16.gguf' to '/content/models/llama_3.2-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 28 key-value pairs and 255 tensors from /content/models/llama_3.2_FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Merged_Model_Bf16
llama_model_loader: - kv   3:                         general.size_label str              = 3.2B
llama_model_loader: - kv   4:                          llama.block_count u32              = 28
llama_model_loader: - kv   5:             

In [14]:
from huggingface_hub import login, create_repo, HfApi

In [15]:
repo_id = "mahmuuud/llama3-3b-finetuned-gguf"
create_repo(repo_id, exist_ok=True)

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/models/llama_3.2-Q4_K_M.gguf",
    path_in_repo="llama3-3b-finetuned.Q4_K_M.gguf",
    repo_id=repo_id,
    commit_message="Add Q4_K_M GGUF"
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /content/models/llama_3.2-Q4_K_M.gguf :   2%|1         | 33.6MB / 2.02GB            

CommitInfo(commit_url='https://huggingface.co/mahmuuud/llama3-3b-finetuned-gguf/commit/3aa670bde6f3d41236fa03f97c2ac8af9a06e284', commit_message='Add Q4_K_M GGUF', commit_description='', oid='3aa670bde6f3d41236fa03f97c2ac8af9a06e284', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mahmuuud/llama3-3b-finetuned-gguf', endpoint='https://huggingface.co', repo_type='model', repo_id='mahmuuud/llama3-3b-finetuned-gguf'), pr_revision=None, pr_num=None)